In [1]:
from experiments.exp_3d.preprocessing import *
from losses.Metrics import metric_dice

from lib.visualize import *
from experiments.exp_3d.inference import Pipeline
import matplotlib.pyplot as plt
import pandas as pd 
import json
import csv

import pydicom
import pydicom_seg
import SimpleITK as sitk
from library_dicom.export_segmentation.ExportSegmentation_Writer import ExportSegmentation_Writer

In [2]:
target_size = (128, 128, 256)
target_spacing = (4.0, 4.0, 4.0)
target_direction = (1,0,0,0,1,0,0,0,1)
model_path = '/media/oncopole/DD 2To/SEGMENTATION/training/20210429-11:17:32/trained_model_20210429-11:17:32.h5'
target_origin = None
from_pp = False

csv_path = '/media/oncopole/DD 2To/SEGMENTATION/inference/inference_dataset.csv'


df = pd.read_csv(csv_path)
dataset = df[['patient_id', 'study', 'study_id', 'ct_img', 'pet_img', 'mask_img']].to_dict('records')
print("")
pipeline = Pipeline(target_size, target_spacing, target_direction, model_path=model_path, target_origin=None, from_pp=False)
print("Pipeline object initialized")


Pipeline object initialized


In [3]:
dataset[0]

{'patient_id': '53011101841003',
 'study': 'ahl',
 'study_id': '1.2.250.1.199.10.90001.93080.20120515093605',
 'ct_img': '/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2/AHL_NIFTI/53011101841003/pet0/1.2.250.1.199.10.90001.93080.20120515093605_nifti_CT.nii',
 'pet_img': '/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2/AHL_NIFTI/53011101841003/pet0/1.2.250.1.199.10.90001.93080.20120515093605_nifti_PT.nii',
 'mask_img': '/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2/AHL_NIFTI/53011101841003/pet0/1.2.250.1.199.10.90001.93080.20120515093605_nifti_mask.nii'}

In [4]:
from library_dicom.dicom_processor.tools.create_mip import *

directory = '/media/oncopole/DD 2To/SEGMENTATION/inference'
nifti_directory = '/media/oncopole/DD 2To/SEGMENTATION/inference/nifti'
vmin = 0 
vmax = 10 
liste_paths_images = []
for count, img_path in enumerate([dataset[0]]) : 
    study_uid = img_path['study_id']
    patient_id = img_path['patient_id']
    pet_img = sitk.ReadImage(img_path['pet_img'], sitk.sitkFloat32)
    pet_spacing = pet_img.GetSpacing()
    pet_size = pet_img.GetSize()
    pet_origin = pet_img.GetOrigin()
    pet_direction = pet_img.GetDirection()
    pet_array = sitk.GetArrayFromImage(pet_img)


    pred_img = pipeline(img_path)
    #save predictions as nifti 

    pred_array = sitk.GetArrayFromImage(pred_img)

    #generate pdf with mip inference
    for type_ in range(1,3):
        filename = inference_pet_projection(pet_array, pred_array, study_uid, patient_id, axis=type_, directory=directory , vmin=vmin, vmax=vmax)
        liste_paths_images.append(filename)
generate_inference_pet_projection_pdf(liste_paths_images, directory, pdf_filename='inference')

(419, 200, 200)


In [ ]:
#Dicom SEG 

segmentation = pred_img
pet_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome/1.2.250.1.59.453.859.92444169.1516.1370594209.2.2.8'
filename = 'sarcome_dicom_seg_v2.dcm'
directory_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome'
export_objet = ExportSegmentation_Writer(segmentation, segmentation_type='img', serie_path = pet_path, mode='dicomseg')
export_objet.generate_dicom(filename = filename, directory_path = directory_path)

In [ ]:
#NIFTI 
#Check here 
img = sitk.GetImageFromArray(np.flip(pred, axis = 0))
img.SetSpacing(pet_spacing)
img.SetOrigin(pet_origin)
img.SetDirection(pet_direction)

directory = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome'
filename = directory + '/'+ 'nifti_mask_prediction_v6.nii'
sitk.WriteImage(img, filename)

In [ ]:
#DICOM RTSTRUCT 
from library_dicom.export_segmentation.ExportSegmentation_Writer import ExportSegmentation_Writer
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.export_segmentation.tools.rtss_writer_tools import *
from skimage.measure import label

#Clean the mask + label 


segmentation = pred_img
segmentation =  sitk.GetArrayFromImage(segmentation)
segmentation = clean_mask(segmentation)
labelled = label(segmentation)
#generate rtstruct file 
pet_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome/1.2.250.1.59.453.859.92444169.1516.1370594209.2.2.8'
export_objet = ExportSegmentation_Writer(labelled, segmentation_type ='matrix', serie_path = pet_path, mode = 'rtstruct')
filename = 'sarcome_rtstruct_v2.dcm'
directory_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome'
export_objet.generate_dicom(filename, directory_path)
